In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from uk_stemmer import UkStemmer

In [32]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

In [33]:
import nlu

In [34]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(suppress=True)
nltk.download('stopwords')
random_state = 8080

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/denisvivdenko/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
df = pd.read_csv("full_data.csv")

In [37]:
df.head()

,crm_customer_id,created,NAME_LOWEST
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [38]:
df.columns = ["customer_id", "date", "category"]

In [39]:
df.head(2)

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві


In [40]:
df.shape

(1412105, 3)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1412105 entries, 0 to 1412104
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   customer_id  1412105 non-null  int64 
 1   date         1412105 non-null  object
 2   category     1412105 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.3+ MB


In [42]:
print(f"number of categories: {df.category.nunique()}")

number of categories: 2129


In [43]:
print(f"number of customers: {df.customer_id.nunique()}")

number of customers: 20114


# NLP feature engineering

1. make all words lowercase
2. remove stopwords
3. preform stemming

In [44]:
df.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [45]:
df.category.unique()[:20]

array(['Петлі меблеві', 'Лотки для столових приборів', 'Інсектициди',
       'Рейлінги для кухні', 'Алюмінієвий профіль', 'Стопори дверні',
       'Зубна паста', 'Куточок алюмінієвий', 'Мішки для сміття',
       'Господарчі рукавички', 'Серветки для прибирання',
       'Аератори для крана', 'Засоби для ванних кімнат', 'Швабри',
       'Засоби для кухні', 'Засоби для догляду за унітазом',
       'Клей побутовий', 'Губки господарчі', 'Пральний порошок',
       'Клей для шпалер'], dtype=object)

In [46]:
"для" in stopwords.words("ukrainian")

True

In [57]:
ndf = df.drop_duplicates(subset=["customer_id", "category"])

In [58]:
print(f"distinct records: {round((ndf.shape[0] * 100) / df.shape[0])}%")

distinct records: 38%


In [59]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,Алюмінієвий профіль


In [60]:
ndf["category"] = ndf["category"].str.lower()

In [61]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [62]:
sum(ndf.category.str.isnumeric())

0

In [63]:
categories = ndf.category.unique()

In [68]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [69]:
without_stopwords_lemma_df = nlu.load('uk.lemma').predict(ndf.head().category, output_level='document')
without_stopwords_lemma_df.lem

lemma download started this may take some time.
Approximate size to download 2.3 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


0                    [петля, меблеві]
1    [лотки, для, столовий, приборів]
2                       [інсектициди]
3              [рейлінги, для, кухня]
4              [алюмінієвий, профіль]
Name: lem, dtype: object

In [67]:
stem_df = nlu.load('uk.stem').predict(ndf.head().category, output_level='document')
stem_df.stem

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


0                    [петлі, меблеві]
1    [лотки, для, столових, приборів]
2                       [інсектициди]
3              [рейлінги, для, кухні]
4              [алюмінієвий, профіль]
Name: stem, dtype: object

In [70]:
for category in categories:
    processed_category = [word for word in word_tokenize(category)
                              if word not in stopwords.words("ukrainian")]
    ndf.loc[ndf["category"] == category, "category"] = ' '.join(processed_category)

In [71]:
lemma_df = nlu.load('uk.lemma').predict(ndf.head().category, output_level='document')
lemma_df.lem

lemma download started this may take some time.
Approximate size to download 2.3 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


0               [петля, меблеві]
1    [лотки, столовий, приборів]
2                  [інсектициди]
3              [рейлінги, кухня]
4         [алюмінієвий, профіль]
Name: lem, dtype: object